## Feature Extraction

In [ ]:
import cv2 
import numpy as np

from google.colab.patches import cv2_imshow

In [ ]:
img1 = cv2.imread('/content/drive/MyDrive/Dive in Learning/Realidad Aumentada/Básico/Colab Notebooks/Robot.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
#cv2_imshow(cv2.resize(img1, (100,100), interpolation=cv2.INTER_AREA))

img2 = cv2.imread('/content/drive/MyDrive/Dive in Learning/Realidad Aumentada/Básico/Colab Notebooks/Foto Robot 1.jpg')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
#cv2_imshow(cv2.resize(img2, (100,100), interpolation=cv2.INTER_AREA))

In [ ]:
orb = cv2.ORB_create(nfeatures=5000)
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)

In [ ]:
print("Keypoints en la Imagen 1: ", len(kp1))
print("Keypoints en la Imagen 2: ", len(kp2))

Keypoints en la Imagen 1:  1675
Keypoints en la Imagen 2:  5000


##Feature Extracion

Brute Force

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
bfmatches = bf.match(des1, des2)
bfmatches = sorted(bfmatches, key=lambda x: x.distance)

In [ ]:
match_img = cv2.drawMatches(img1, kp1, img2, kp2, bfmatches[:50], None)
cv2_imshow(match_img)

Output hidden; open in https://colab.research.google.com to view.

FLANN - KDTREE

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 1)
search_params = dict(checks = 50)

In [ ]:
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [ ]:
flann_matches = flann.knnMatch(np.asarray(des1, np.float32), np.asarray(des2, np.float32), k=2)

In [ ]:
# store all the good matches as per Lowe's ratio test.
flann_good_matches = []

for m1, m2 in flann_matches:
  if m1.distance < 0.5 * m2.distance:
    flann_good_matches.append([m1])

In [ ]:
flann_matches =cv2.drawMatchesKnn(img1, kp1, img2, kp2, flann_good_matches, None, flags=2)
cv2_imshow(flann_matches)

Output hidden; open in https://colab.research.google.com to view.

## Homography

In [ ]:
print("BF Matches: ", len(bfmatches))
print("FLANN Matches: ", len(flann_good_matches))

BF Matches:  388
FLANN Matches:  10


In [ ]:
if (len(flann_good_matches) > len(bfmatches[:50])):
  matches = flann_good_matches
else:
  matches = bfmatches[:50]

src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

# Finally find the homography matrix
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

matchesMask = mask.ravel().tolist()
h,w = img1.shape

# matchesMask = mask.ravel().tolist()
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

dst = cv2.perspectiveTransform(pts,M)
plines = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

In [ ]:
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv2.drawMatches(img1, kp1, plines, kp2, matches, None, **draw_params)
cv2_imshow(img3)

Output hidden; open in https://colab.research.google.com to view.